Source: https://github.com/BlackPianoCat/EasyOpenMM/tree/main

# Imports

In [5]:
import copy
import numpy as np
import time
from utils import *
import openmm as mm
import openmm.unit as u
from tqdm import tqdm
from sys import stdout
from mdtraj.reporters import HDF5Reporter
from openmm.app import PDBFile, PDBxFile, ForceField, Simulation, PDBReporter, PDBxReporter, DCDReporter, StateDataReporter, CharmmPsfFile,  DCDFile
import random
import pyvista as pv
import mdtraj as md
from hilbert import decode, encode
from initial_structures_defs import *
from model import Model

# Initial structure generation functions

# Simulation

In [13]:
# 0. Generate some initial structure
N_beads=100
points = helisa(N_beads)
write_mmcif(points,'init_struct.cif')
generate_psf(N_beads,'LE_init_struct.psf')

# 1. Define System
pdb = PDBxFile('init_struct.cif')
forcefield = ForceField('forcefields/classic_sm_ff.xml')
system = forcefield.createSystem(pdb.topology, nonbondedCutoff=1*u.nanometer)
integrator = mm.LangevinIntegrator(310, 0.05, 100 * mm.unit.femtosecond)

# 2. Define the forcefield
# 2.1. Harmonic bond borce between succesive beads
bond_force = mm.HarmonicBondForce()
system.addForce(bond_force)
for i in range(system.getNumParticles() - 1):
    bond_force.addBond(i, i + 1, 0.1, 300000.0)
bond_force.addBond(10, 70, 0.001, 0.001) # connect bead 10 with bead 70

#2.2. Harmonic angle force between successive beads so as to make chromatin rigid
angle_force = mm.HarmonicAngleForce()
system.addForce(angle_force)
for i in range(system.getNumParticles() - 2):
    angle_force.addAngle(i, i + 1, i + 2, np.pi, 0.001)
    
# 3. Minimize energy
simulation = Simulation(pdb.topology, system, integrator)
simulation.reporters.append(StateDataReporter(stdout, 10, step=True, totalEnergy=True, potentialEnergy=True, temperature=True))
simulation.reporters.append(DCDReporter('stochastic_LE.dcd', 10))
simulation.context.setPositions(pdb.positions)
simulation.minimizeEnergy(tolerance=0.001)
state = simulation.context.getState(getPositions=True)
PDBxFile.writeFile(pdb.topology, state.getPositions(), open('minimized.cif', 'w')) # save minimized file

# 4. Run md simulation
simulation.context.setVelocitiesToTemperature(310, 0)
simulation.step(10000)
state = simulation.context.getState(getPositions=True)
PDBxFile.writeFile(pdb.topology, state.getPositions(), open('after_sim.cif', 'w')) # save minimized file
PDBFile.writeFile(pdb.topology, state.getPositions(), open('after_sim.pdb', 'w')) # save minimized file
df = DCDFile(open("after_sim.dcd", "wb"),pdb.topology, dt = 100 * mm.unit.femtosecond)
df.writeModel(state.getPositions(), pdb.topology.getUnitCellDimensions(), pdb.topology.getPeriodicBoxVectors())

#"Step","Potential Energy (kJ/mole)","Total Energy (kJ/mole)","Temperature (K)"
10,107.27322387695312,316.13366567716,169.15917313518588
20,82.96114349365234,320.29261162132025,192.21828011759123
30,89.17096710205078,337.1468686237931,200.8393647802227
40,94.56158447265625,336.2572063356638,195.75287303016904
50,91.58721923828125,357.9594160877168,215.7387975285268
60,82.23358154296875,343.2710475176573,211.4181197788637
70,81.62876892089844,351.0761366188526,218.22942406125458
80,96.01492309570312,356.9387115314603,211.32605065211126
90,87.42857360839844,379.08236931450665,236.2147398432575
100,103.54850006103516,391.5979905053973,233.29555949226028
110,87.23733520507812,406.9284926354885,258.9226154241774
120,89.27845764160156,412.5447427332401,261.81816440326367
130,103.89022827148438,423.42676820605993,258.7973884184208
140,104.93695831298828,419.017266407609,254.37830523289568
150,127.93624877929688,434.52535669505596,248.31107097290794
160,106.31491088867188,454.4865112155676,281

# Visualization

In [14]:
traj = md.load("after_sim.cif")

positions = traj.xyz

mesh = pv.PolyData(positions[0])

# Create PyVista plotter
plotter = pv.Plotter(notebook=True)

# Add mesh to the plotter
plotter.add_mesh(mesh, color="blue", point_size=5)

# Create lines between consecutive points
lines = pv.lines_from_points(positions[0])

# Add lines to the plotter
plotter.add_mesh(lines, color="red", line_width=2)

# Show the plotter using Trame's notebook backend
plotter.show(jupyter_backend='trame')

Widget(value='<iframe src="http://localhost:52478/index.html?ui=P_0x1f9595e7a00_1&reconnect=auto" class="pyvis…

### Testing initial structure generation function

In [15]:
positions = cube_outer(1000)

mesh = pv.PolyData(positions)

# Create PyVista plotter
plotter = pv.Plotter(notebook=True)

# Add mesh to the plotter
plotter.add_mesh(mesh, color="blue", point_size=5)

# Create lines between consecutive points
lines = pv.lines_from_points(positions)

# Add lines to the plotter
plotter.add_mesh(lines, color="red", line_width=2)

# Show the plotter using Trame's notebook backend
plotter.show(jupyter_backend='trame')

Widget(value='<iframe src="http://localhost:52478/index.html?ui=P_0x1f95806f820_2&reconnect=auto" class="pyvis…

In [25]:
positions = hilbert_curve3d(4000)

mesh = pv.PolyData(positions)

# Create PyVista plotter
plotter = pv.Plotter(notebook=True)

# Add mesh to the plotter
plotter.add_mesh(mesh, color="blue", point_size=5)

# Create lines between consecutive points
lines = pv.lines_from_points(positions)

# Add lines to the plotter
plotter.add_mesh(lines, color="red", line_width=2)

# Show the plotter using Trame's notebook backend
plotter.show(jupyter_backend='trame')

c:\Users\Micha\anaconda3\envs\wb\lib\site-packages\pyvista\core\utilities\points.py:52: UserWarning: Points is not a float type. This can cause issues when transforming or applying filters. Casting to ``np.float32``. Disable this by passing ``force_float=False``.
  warnings.warn(


Widget(value='<iframe src="http://localhost:52478/index.html?ui=P_0x1f947e019d0_8&reconnect=auto" class="pyvis…

# Testing multiple initial structures

In [6]:
initial_structure_gen_list = [line,random_walk,random_walk2,hilbert_curve2d, hilbert_curve3d, helisa, sphere, cube, cube_outer]

In [4]:
import os
def make_directory_if_not_exists(directory_path):
    if not os.path.exists(directory_path):
        os.makedirs(directory_path)

for f in initial_structure_gen_list:
    make_directory_if_not_exists(f'initial_structures_tests/{f.__name__}')

In [7]:
for f in initial_structure_gen_list:
    #run_simulation_on_initial_structure(f,100)
    model = Model(200, 30, struct=f)
    model.run_simulation()

#"Step","Potential Energy (kJ/mole)","Total Energy (kJ/mole)","Temperature (K)"
10,53208137728.0,141618081773.7008,35622352360.375435
20,89351815168.0,134868639910.91449,18339751108.53383
30,91468922880.0,131190356909.92047,16004658011.567217
40,62731042816.0,125596709612.61945,25329989269.55018
50,53390270464.0,118422230171.04524,26202837343.417084
60,57341919232.0,112642005783.71631,22281647047.37522
70,60937121792.0,107708332860.00253,18845171535.5078
80,49437474816.0,102759470952.67386,21484629986.388287
90,53157068800.0,97533127191.5888,17880110721.172234
100,46011052032.0,92842999428.44623,18869643566.524784
110,37091704832.0,88496147352.8218,20712004561.600273
120,40207192064.0,83927729594.26035,17615986641.535343
130,44624707584.0,79590508729.82193,14088506699.37572
140,42333683712.0,76080762648.26242,13597456145.651777
150,35205926912.0,72545831757.22069,15045086408.060265
160,29538193408.0,68592872883.42163,15736007624.615664


/Users/krzysztof/w_badawcze/project_repo_2/Bioinformatics_Project/test_test/model.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_chr1['middle1'] = (df_chr1['end1'] + df_chr1['start1'])/2
/Users/krzysztof/w_badawcze/project_repo_2/Bioinformatics_Project/test_test/model.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_chr1['middle2'] = (df_chr1['end2'] + df_chr1['start2'])/2


170,30848057344.0,65169866152.34036,13829027720.95569
180,35719962624.0,62217862201.49159,10676598947.640095
190,26267734016.0,59245250466.238174,13287382133.770128
200,26134462464.0,56131495893.19504,12086478575.627028
210,27235938304.0,53361792943.56297,10526693685.106094
220,28022374400.0,50986342564.32173,9252698600.4997
230,26678517760.0,48630830714.03332,8845079992.015173
240,24380145664.0,46146393909.37341,8770110341.426008
250,21235167232.0,44059573194.364685,9196465854.421572
260,23318976512.0,41941345912.64709,7503370935.64966
270,17251221504.0,39773696486.18254,9074811080.395218
280,16051187712.0,37600541700.95915,8682718773.539295
290,17420109824.0,35714481397.93324,7371213243.63033
300,19541819392.0,34145057541.863068,5883972675.155149
310,18065006592.0,32771941468.320065,5925754415.527695
320,15507841024.0,31176351857.514107,6313194968.037547
330,12484798464.0,29488840875.866676,6851310640.273368
340,13415293952.0,27889607593.924072,5832026094.94561
350,14289221632.0,2658

/Users/krzysztof/w_badawcze/project_repo_2/Bioinformatics_Project/test_test/model.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_chr1['middle1'] = (df_chr1['end1'] + df_chr1['start1'])/2
/Users/krzysztof/w_badawcze/project_repo_2/Bioinformatics_Project/test_test/model.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_chr1['middle2'] = (df_chr1['end2'] + df_chr1['start2'])/2


#"Step","Potential Energy (kJ/mole)","Total Energy (kJ/mole)","Temperature (K)"
10,3234.85009765625,3757.281466199085,210.49933347685464
20,3277.818359375,3789.3846419900656,206.1215463000559
30,3392.74853515625,3833.6148621067405,177.63494606824767
40,3486.8349609375,3863.4547293726355,151.74856450700128
50,3485.135498046875,3887.8359451293945,162.2570557700948
60,3430.96875,3939.242024162784,204.7947192255786
70,3432.78759765625,3962.1633559316397,213.29738408814063
80,3395.935302734375,3970.6687364745885,231.5730103776483
90,3411.849853515625,3957.1242330260575,219.70329570579858
100,3405.4072265625,3966.0318459519185,225.88825215710986
110,3404.87646484375,3993.3917915821075,237.12604464174757
120,3416.650146484375,3996.437047773972,233.60916596658086
130,3421.223876953125,4010.6347722411156,237.48688932706273
140,3393.0517578125,4022.920730918646,253.78835766773744
150,3402.5673828125,4033.0444962978363,254.03339108683898
160,3429.17626953125,4051.5337805747986,250.76182087684074


...

#"Step","Potential Energy (kJ/mole)","Total Energy (kJ/mole)","Temperature (K)"
10,3382.5087890625,3928.7927739545703,220.11008839227557
20,3459.819091796875,3960.7705562263727,201.84459761821805
30,3567.84423828125,4002.4905078709126,175.1287452398134
40,3676.61474609375,4027.9809865280986,141.57335081925564
50,3652.306884765625,4056.065992053598,162.68361374697895
60,3571.95556640625,4111.618430703878,217.4422901781536
70,3542.475341796875,4152.372023124248,245.7410726833706
80,3523.84814453125,4181.963475331664,265.1694496652881
90,3579.724853515625,4177.443863481283,240.83441535134455
100,3589.1552734375,4185.308292547241,240.20344245949678
110,3630.475341796875,4206.266453765333,231.99917269385716
120,3653.648681640625,4226.170591983944,230.68193792479548
130,3677.020263671875,4222.774920013268,219.89681002918894
140,3630.56103515625,4238.939776130021,245.12946041823784
150,3628.73974609375,4272.0102993920445,259.18815535954207
160,3678.347900390625,4305.992412656546,252.892072428

/Users/krzysztof/w_badawcze/project_repo_2/Bioinformatics_Project/test_test/model.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_chr1['middle1'] = (df_chr1['end1'] + df_chr1['start1'])/2
/Users/krzysztof/w_badawcze/project_repo_2/Bioinformatics_Project/test_test/model.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_chr1['middle2'] = (df_chr1['end2'] + df_chr1['start2'])/2


#"Step","Potential Energy (kJ/mole)","Total Energy (kJ/mole)","Temperature (K)"
10,3495.16455078125,4048.0034573711455,222.75121358396103
20,3587.66552734375,4068.750309471041,193.83986506420783
30,3677.97802734375,4107.504502102733,173.06586489937308
40,3759.32568359375,4146.636376213282,156.0561780052949
50,3749.557373046875,4223.520457267761,190.97037300690081
60,3689.945556640625,4231.526785336435,218.21524228535424
70,3657.34912109375,4244.782454818487,236.6900853523409
80,3660.93896484375,4275.722257101908,247.7099979926607
90,3693.40380859375,4280.285526074469,236.46782677697112
100,3744.5595703125,4299.287806816399,223.51246704530067
110,3792.8720703125,4332.781913146377,217.54180337815217
120,3816.119384765625,4350.190132245421,215.18910069957838
130,3767.6552734375,4384.5211216285825,248.54910655735412
140,3790.1767578125,4392.041590601206,242.50486049210775
150,3753.764892578125,4397.716012948193,259.46237111996646
160,3782.59521484375,4436.580889865756,263.5055185899617
170

/Users/krzysztof/w_badawcze/project_repo_2/Bioinformatics_Project/test_test/model.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_chr1['middle1'] = (df_chr1['end1'] + df_chr1['start1'])/2
/Users/krzysztof/w_badawcze/project_repo_2/Bioinformatics_Project/test_test/model.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_chr1['middle2'] = (df_chr1['end2'] + df_chr1['start2'])/2


#"Step","Potential Energy (kJ/mole)","Total Energy (kJ/mole)","Temperature (K)"
10,3570.091552734375,4112.66783535108,218.6161718614364
20,3619.50390625,4117.759585928172,200.758405387966
30,3722.013916015625,4147.048793889582,171.25609962210478
40,3796.71826171875,4168.950695453212,149.98080880579283
50,3754.314697265625,4210.274432938546,183.71641947752448
60,3748.058349609375,4221.524797081947,190.77026690532466
70,3792.07275390625,4251.621820850298,185.16264167185255
80,3751.23779296875,4302.262259265408,222.0201348330327
90,3761.431640625,4316.396673616022,223.60787766899278
100,3765.32861328125,4338.812196070794,231.06941735490318
110,3775.734130859375,4338.18129144609,226.62259494424958
120,3785.89013671875,4392.005940580741,244.2176722247793
130,3791.72509765625,4397.46427282691,244.06592006503732
140,3784.109619140625,4419.696149729192,256.09209000778094
150,3764.180908203125,4425.8554016742855,266.60351625286205
160,3809.737548828125,4456.076114527881,260.42432647118045
170,3

In [8]:
plotter = pv.Plotter(shape=(len(initial_structure_gen_list), 2), notebook=True, window_size=(1000,2000))
for i,f in enumerate(initial_structure_gen_list):
    traj = md.load(f'initial_structures_tests/{f.__name__}/after_sim.cif')
    positions = traj.xyz
    mesh = pv.PolyData(positions[0])
    plotter.subplot(i,1)
    plotter.add_text(f"{f.__name__}, after simulation", font_size=10) 
    plotter.add_mesh(mesh, color="blue", point_size=5)
    lines = pv.lines_from_points(positions[0])
    plotter.add_mesh(lines, color="red", line_width=2)

    traj = md.load(f'initial_structures_tests/{f.__name__}/init_struct.cif')
    positions = traj.xyz
    mesh = pv.PolyData(positions[0])
    plotter.subplot(i,0)
    plotter.add_text(f"{f.__name__}, initial structure", font_size=10) 
    plotter.add_mesh(mesh, color="blue", point_size=5)
    lines = pv.lines_from_points(positions[0])
    plotter.add_mesh(lines, color="red", line_width=2)
plotter.show(jupyter_backend='trame')

Widget(value='<iframe src="http://localhost:65091/index.html?ui=P_0x7fb07985d040_1&reconnect=auto" class="pyvi…